<a href="https://colab.research.google.com/github/BoWarburton/DS-Unit-2-Kaggle-Challenge/blob/master/Bo_Warburton_LS_DS_223_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 2, Module 3*

---

# Cross-Validation


## Assignment
- [X] [Review requirements for your portfolio project](https://lambdaschool.github.io/ds/unit2), then submit your dataset.
- [X] Continue to participate in our Kaggle challenge. 
- [X] Use scikit-learn for hyperparameter optimization with RandomizedSearchCV.
- [X] Submit your predictions to our Kaggle competition. (Go to our Kaggle InClass competition webpage. Use the blue **Submit Predictions** button to upload your CSV file. Or you can use the Kaggle API to submit your predictions.)
- [X] Commit your notebook to your fork of the GitHub repo.


You won't be able to just copy from the lesson notebook to this assignment.

- Because the lesson was ***regression***, but the assignment is ***classification.***
- Because the lesson used [TargetEncoder](https://contrib.scikit-learn.org/categorical-encoding/targetencoder.html), which doesn't work as-is for _multi-class_ classification.

So you will have to adapt the example, which is good real-world practice.

1. Use a model for classification, such as [RandomForestClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html)
2. Use hyperparameters that match the classifier, such as `randomforestclassifier__ ...`
3. Use a metric for classification, such as [`scoring='accuracy'`](https://scikit-learn.org/stable/modules/model_evaluation.html#common-cases-predefined-values)
4. If you’re doing a multi-class classification problem — such as whether a waterpump is functional, functional needs repair, or nonfunctional — then use a categorical encoding that works for multi-class classification, such as [OrdinalEncoder](https://contrib.scikit-learn.org/categorical-encoding/ordinal.html) (not [TargetEncoder](https://contrib.scikit-learn.org/categorical-encoding/targetencoder.html))



## Stretch Goals

### Reading
- Jake VanderPlas, [Python Data Science Handbook, Chapter 5.3](https://jakevdp.github.io/PythonDataScienceHandbook/05.03-hyperparameters-and-model-validation.html), Hyperparameters and Model Validation
- Jake VanderPlas, [Statistics for Hackers](https://speakerdeck.com/jakevdp/statistics-for-hackers?slide=107)
- Ron Zacharski, [A Programmer's Guide to Data Mining, Chapter 5](http://guidetodatamining.com/chapter5/), 10-fold cross validation
- Sebastian Raschka, [A Basic Pipeline and Grid Search Setup](https://github.com/rasbt/python-machine-learning-book/blob/master/code/bonus/svm_iris_pipeline_and_gridsearch.ipynb)
- Peter Worcester, [A Comparison of Grid Search and Randomized Search Using Scikit Learn](https://blog.usejournal.com/a-comparison-of-grid-search-and-randomized-search-using-scikit-learn-29823179bc85)

### Doing
- Add your own stretch goals!
- Try other [categorical encodings](https://contrib.scikit-learn.org/categorical-encoding/). See the previous assignment notebook for details.
- In additon to `RandomizedSearchCV`, scikit-learn has [`GridSearchCV`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html). Another library called scikit-optimize has [`BayesSearchCV`](https://scikit-optimize.github.io/notebooks/sklearn-gridsearchcv-replacement.html). Experiment with these alternatives.
- _[Introduction to Machine Learning with Python](http://shop.oreilly.com/product/0636920030515.do)_ discusses options for "Grid-Searching Which Model To Use" in Chapter 6:

> You can even go further in combining GridSearchCV and Pipeline: it is also possible to search over the actual steps being performed in the pipeline (say whether to use StandardScaler or MinMaxScaler). This leads to an even bigger search space and should be considered carefully. Trying all possible solutions is usually not a viable machine learning strategy. However, here is an example comparing a RandomForestClassifier and an SVC ...

The example is shown in [the accompanying notebook](https://github.com/amueller/introduction_to_ml_with_python/blob/master/06-algorithm-chains-and-pipelines.ipynb), code cells 35-37. Could you apply this concept to your own pipelines?

What is the target?
What type of problem? (Regression = 'How much/many' & Classification = 'Which one')
What type of model would I use?
Which features do I want to use?
Make a prediction (Train, test split)


# Setup

In [0]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Kaggle-Challenge/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'

In [11]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Merge train_features.csv & train_labels.csv
train = pd.merge(pd.read_csv(DATA_PATH+'waterpumps/train_features.csv'), 
                 pd.read_csv(DATA_PATH+'waterpumps/train_labels.csv'))

# Read test_features.csv & sample_submission.csv
test = pd.read_csv(DATA_PATH+'waterpumps/test_features.csv')
sample_submission = pd.read_csv(DATA_PATH+'waterpumps/sample_submission.csv')

train.shape, test.shape

((59400, 41), (14358, 40))

In [0]:
import numpy as np

# Split train into train & val
train, val = train_test_split(train, train_size=0.80, test_size=0.20, 
                              stratify=train['status_group'], random_state=42)

def wrangle(X):
    
    # Prevent SettingWithCopyWarning
    X = X.copy()
    
    # Treat very small latitude values as zero.
    X['latitude'] = X['latitude'].replace(-2e-08, 0)
    
    # Replace zeros with nulls, impute missing values later
    # Also create a "missing indicator" column, because the fact that
    # values are missing may be a predictive signal.
    cols_with_zeros = ['longitude', 'latitude', 'construction_year', 
                       'gps_height', 'population']
    for col in cols_with_zeros:
        X[col] = X[col].replace(0, np.nan)
        X[col+'_MISSING'] = X[col].isnull()
            
    # Drop duplicate columns
    duplicates = ['quantity_group', 'payment_type']
    X = X.drop(columns=duplicates)
    
    # Drop columns that never vary or are random
    # recorded_by (never varies) and id (always varies, random)
    unusable_variance = ['recorded_by', 'id']
    X = X.drop(columns=unusable_variance)
    
    # Convert date_recorded to datetime
    X['date_recorded'] = pd.to_datetime(X['date_recorded'], infer_datetime_format=True)
    
    # Extract components from date_recorded, then drop the original column
    X['year_recorded'] = X['date_recorded'].dt.year
    X['month_recorded'] = X['date_recorded'].dt.month
    X['day_recorded'] = X['date_recorded'].dt.day
    X = X.drop(columns='date_recorded')
    
    # Engineer feature: how many years from construction_year to date_recorded
    X['years'] = X['year_recorded'] - X['construction_year']
    X['years_MISSING'] = X['years'].isnull()
    
    # return the wrangled dataframe
    return X

train = wrangle(train)
val = wrangle(val)
test = wrangle(test)

In [13]:
test.shape, train.shape, val.shape

((14358, 45), (47520, 46), (11880, 46))

In [0]:
# The status_group column is the target
target = 'status_group'

# Get a dataframe with all train columns except the target
train_features = train.drop(columns=[target])

# Get a list of the numeric features
numeric_features = train_features.select_dtypes(include='number').columns.tolist()

# Get a series with the cardinality of the nonnumeric features
cardinality = train_features.select_dtypes(exclude='number').nunique()

# Get a list of all categorical features with cardinality <= 50
categorical_features = cardinality[cardinality <= 50].index.tolist()

# Combine the lists
features = numeric_features + categorical_features

In [18]:
# Look at cardinality
cardinality.sort_values(ascending=False)

wpt_name                     30661
subvillage                   17231
scheme_name                   2563
ward                          2082
installer                     1929
funder                        1716
lga                            124
region                          21
extraction_type                 18
extraction_type_group           13
scheme_management               12
management                      12
source                          10
basin                            9
water_quality                    8
waterpoint_type                  7
payment                          7
extraction_type_class            7
source_type                      7
quality_group                    6
waterpoint_type_group            6
management_group                 5
quantity                         5
source_class                     3
public_meeting                   2
permit                           2
longitude_MISSING                2
latitude_MISSING                 2
population_MISSING  

In [20]:
high_cardinality = cardinality[cardinality > 50].index.tolist()
high_cardinality

['funder', 'installer', 'wpt_name', 'subvillage', 'lga', 'ward', 'scheme_name']

# Use scikit-learn for hyperparameter optimization with RandomizedSearchCV

In [0]:
# Make X features and y target
X_train = train[features]
X_val = val[features]
X_test = test[features]
y_train = train[target]
y_val = val[target]

In [0]:
import category_encoders as ce
from sklearn.feature_selection import f_regression, SelectKBest
from sklearn.impute import SimpleImputer
from sklearn.linear_model import Ridge
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

In [26]:
# Using Ordinal encoding
pipeline = make_pipeline(
    ce.OrdinalEncoder(),
    SimpleImputer(),
    RandomForestClassifier(n_jobs=-1, random_state=0)
)

# Set hyperparameters to search
p_dist = {
    'simpleimputer__strategy': ['mean','median'],
    'randomforestclassifier__n_estimators': range(1, 300, 20),
    'randomforestclassifier__max_depth': [20, 30, None],
#    'randomforestclassifier__max_features': uniform(0, 1),
    'randomforestclassifier__random_state': range(120,150,2),
}

search = RandomizedSearchCV(
    pipeline,
    param_distributions=p_dist,
    n_iter=5,
    cv=5,
    scoring='accuracy',
    verbose=10,
    return_train_score=True,
    n_jobs=-1
)

# Fit on train
search.fit(X_train, y_train)

Fitting 5 folds for each of 5 candidates, totalling 25 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    3.0s
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:    5.0s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:   27.1s
[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed:   52.6s
/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=-1)]: Done  21 tasks      | elapsed:  2.2min
[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed:  2.5min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed:  2.5min finished


RandomizedSearchCV(cv=5, error_score=nan,
                   estimator=Pipeline(memory=None,
                                      steps=[('ordinalencoder',
                                              OrdinalEncoder(cols=None,
                                                             drop_invariant=False,
                                                             handle_missing='value',
                                                             handle_unknown='value',
                                                             mapping=None,
                                                             return_df=True,
                                                             verbose=0)),
                                             ('simpleimputer',
                                              SimpleImputer(add_indicator=False,
                                                            copy=True,
                                                            fill_value=None,


In [27]:
from pprint import pprint
print("Best Hyperparameters:") 
pprint(search.best_params_)
print("Cross-validation Accuracy:", search.best_score_)

Best Hyperparameters:
{'randomforestclassifier__max_depth': 20,
 'randomforestclassifier__n_estimators': 261,
 'randomforestclassifier__random_state': 138,
 'simpleimputer__strategy': 'median'}
Cross-validation Accuracy: 0.8044823232323232


In [0]:
# Set pipeline
pipeline = search.best_estimator_

In [29]:
# Pipeline score
pipeline.score(X_train, y_train)

0.9539141414141414

In [0]:
# Submit 

y_pred = pipeline.predict(X_test)

submission = sample_submission.copy()
submission['status_group'] = y_pred
submission.to_csv('bowarburton_v14.csv', index=False)

In [0]:
from google.colab import files
files.download('bowarburton_v14.csv')

# Installers

In [0]:
!pip install category_encoders
!pip install LambdaData

In [0]:
raw_train = pd.merge(
    pd.read_csv(DATA_PATH+'waterpumps/train_features.csv'), 
    pd.read_csv(DATA_PATH+'waterpumps/train_labels.csv'),
)
raw_test = pd.read_csv(DATA_PATH+'waterpumps/test_features.csv')
sample_submission = pd.read_csv(DATA_PATH+'waterpumps/sample_submission.csv')

# Wrangle

In [0]:
def wrangle(X):
    X = X.copy()
    X = X.drop(columns = ['recorded_by', 'id'])
    X['latitude'] = X['latitude'].replace(-2e-08, np.nan)
    cols_with_zeros = ['construction_year', 'longitude', 'latitude', 'gps_height']
    for col in cols_with_zeros:
        X[col] = X[col].replace(0, np.nan)
    return X

In [0]:
train = wrangle(raw_train)
test = wrangle(raw_test)
high_cardinality = ['display_address', 'street_address', 'description']


# Train/test/val split

In [0]:
X_train, X_val = train_test_split(
    raw_train,
    train.size = 0.8,
    test_size = 0.2,
    stratify = raw_train['status_group'],
    random_state = 37
)

In [0]:
X_train, X_val = train_test_split(train, random_state=42)
y_train = X_train['status_group']
y_val = X_val['status_group']
X_train = X_train.drop(columns=['status_group'])
X_val = X_val.drop(columns=['status_group'])

# Accuracy function

In [0]:
def acc_score(model, X, y):
    return f'Accuracy: {100 * model.score(X_val, y_val) :.2f}%'

# Model training and validation

In [0]:
# Tweak randomforest parameters here to get good validation accuracy

pipeline = make_pipeline(
    OrdinalEncoder(),
    SimpleImputer(strategy='most_frequent'),
    StandardScaler(),
    RandomForestClassifier(
        criterion='entropy', 
        max_depth=20, 
        max_features=6,
        n_jobs=-1, 
        random_state=42,
        n_estimators=999,
    ),
).fit(X_train, y_train)

print('Validation', acc_score(pipeline, X_val, y_val))

# Data consolidation and model re-training

In [0]:
pipeline = make_pipeline(
    OrdinalEncoder(),
    SimpleImputer(strategy='most_frequent'),
    StandardScaler(),
    RandomForestClassifier(
        criterion='entropy', 
        max_depth=20, 
        max_features=6,
        n_jobs=-1, 
        random_state=42,
        n_estimators=999,
    ),
).fit(X_train, y_train)

print('Validation', acc_score(pipeline, X_val, y_val))

pipeline = make_pipeline(
    OrdinalEncoder(),
    SimpleImputer(),
    RandomForestClassifier(
        n_jobs=-1, 
        random_state=42,
    ),
).fit(star_cat_row(X_train, X_val), star_cat_row(y_train, y_val));

# Submission output

In [0]:
y_pred = pipeline.predict(X_test)
submission = sample_submission.copy()
submission['status_group'] = y_pred
submission['status_group'] = submission['status_group']
submission.to_csv('submission-01.csv', index=False)
submission['status_group'].value_counts()

### BONUS: Stacking!

Here's some code you can use to "stack" multiple submissions, which is another form of ensembling:

```python
import pandas as pd

# Filenames of your submissions you want to ensemble
files = ['submission-01.csv', 'submission-02.csv', 'submission-03.csv']

target = 'status_group'
submissions = (pd.read_csv(file)[[target]] for file in files)
ensemble = pd.concat(submissions, axis='columns')
majority_vote = ensemble.mode(axis='columns')[0]

sample_submission = pd.read_csv('sample_submission.csv')
submission = sample_submission.copy()
submission[target] = majority_vote
submission.to_csv('my-ultimate-ensemble-submission.csv', index=False)
```